# Locality-sensitive hashing for detecting coreferences

Problem: We want to find mentions that cold refer to the same entity. Currently we compare each mention with each other, and if mention 1 contains mention 0 as a word, then we consider mention 0 and mention 1 potential coreferences. This approach does not scale well with the number of mentions.

Solution: try out how to use locality-sensitive hashing to reduce the number of comparisons to make. While there is optimized software available to do this, I think that a good start is a solution without external dependencies: no need to check compatibility with other requirements, and data are already preprocessed which should make the task computationally simple. 

How does LSH work?
1. Shingling: convert text to sparse vectors. I will start with shingling at the word level and think about alternatives later.
    - One-hot encoding
    - Define vocabulary of all shingles
2. MinHashing: create signatures based on randomly reordering the vocabulary and recording the first shingle that is in mention $i$.
3. Cut the signature into bands and assign--using a hash function--all mentions to buckets. 
    - More bands means larger buckets
    - Need to use the same function for the same band $j$ for all mentions. Can use different functions for different bands, but not required. 

Next steps, 4/1/23

1. Finish signature with numpy
    - code below to LSH
    - tests 
    - correctness of lsh after integration
    - fix the (memory?) issue 
1. write tests 
    - [x] correctness of cols_to_int
    - [ ] notebook with speed tests for different sorting options -- not for now
    - [x] correctness of coreference classification -- precision and recall for the clustering 
    - [x] optimize for current use case (shingle size, size of bands, ...)
2. Understand better
    - [ ] what do shingles do? smaller ->?
    - [ ] what does band size do? how does it interact with shingle size? does one compensate for the other? does one scale better than the other? (for optimization)
2. Document 
    - [ ] write what the functions/classes do
    - [ ] `cols_to_int`:  
        - write down/explain the idea/mechanics
        - how far can we go with until overflow error? what to do in this case?
            - advise user to change the size of the bands (?)
            - switch to string operation, which is (much) slower but should still work? 
3. Integrate with REL
    - tests?
        - how to handle empty input, input where only one item is repeated (ie "EEC"), ... 
    - update script for msmarco
    - move the changed functions for signature to REL
4. Optimize further?
    - alternative for minhashing? improve speed or effectiveness
    - the advantage of the while loop is the smaller memory taken. are there other ways to speed it up?
    - or reduce signature size/shingle size for the signature to buy some speed at the expense of some correctness? how strong would the impact be?
    - should the performance not be assessed on the universe of mentions, and not only the coreference mentions?
    - change the band length for the performance testing?

## Speeding up the hashing
The approach with min hashing is not feasible for high-dimensional data sets because it is expensive to randomly shuffle large arrays and/or store large binary arrays in memory. But there are alternatives. One is with random projections, as discussed in the wikipedia page, in [this video](https://www.youtube.com/watch?v=Arni-zkqMBA) and in [this book](http://infolab.stanford.edu/~ullman/mmds/ch3n.pdf) (section 3.7.2 Random Hyperplanes and the Cosine Distance).

What is the idea?
- Use the cosine similarity between two binary vectors with the cosine distance
- Suppose we pick a hyperplane through the origin. We do so by choosing a vector $v$ that is perpendicular to the hyperplane. 
    - Then, either two vectors $x$ and $y$ lie on the same side of the hyperplane or they do not. 
- Then, choose a random vector $v_f$. 
    - we can build a hashing function $f$ that assigns the same value to $x$ and $y$ if they lie on the same side of the hyperplane that $v$ is perpendicular to (the dot product $x.v$ and $y.v$ will be informative about this.)
    - (the angle $\theta$ between $x$ and $y$ will determine the probability that $x$ and $y$ are on the same side of a given hyperplane; see book for intuition)
    - the family $F$ of functions defined by vectors $v_f$ is locality-sensitive for the cosine distance. And this is very similar to the Jaccard-distance family, up to some scaling factor. 
- We can then build sketches (see the code [here](https://github.com/brandonrobertz/SparseLSH/blob/11f381560a94c8d74af55b3db5e8db1bbddfc212/sparselsh/lsh.py#L140)) by using random vector whose elements are either +1 or -1.
    - consider a random vector $v$ where $v_i \in \{-1, 1\} \forall i$.
    - we calculate the dot product for a random vector $v$ and vector $x$.
    - the dot product is the difference between the sum of all elements $x[i]$ for $i: v[i] = 1$, and the sum of all elements $x[i]$ for $i: v[i] = -1$. 
    - We repeat this for multiple vectors $v$ and store whether the dot product is positive or negative (again by $+1$ or $-1$). Since a dot product of $0$ is unlikely, we can handle such cases arbitrarily. 
    - The result of this is called a **sketch** (which is the same as a signature, see [p. 49 here](https://web.stanford.edu/class/cs246/slides/04-lsh_theory.pdf))
- I do not understand the example 3.22: does it imply that we have to take a large number of random vectors? what is the 
- p. 99: cosine distance makes sense in spaces [...] where points are vectors with integer components or Boolean components -- thus, we can use it here. 

Now I am not sure how SparseLSH implements the whole thing. They have different distance options, but the hashing is, as far as I understand, the same for all options. What is the theory behind this? Can I just use this hashing instead of my hashing, and then continue with the signature as before (which is essentially the Jaccard distance??)
    - or is this perhaps what is discussed [here](https://ai.stackexchange.com/questions/37199/clustering-by-using-locality-sensitive-hashing-after-random-projection)?
    - in fact, the slides from Stanford seem to imply that using cosine distance for LSH in the same way min-hashing was used for the Jaccard distance.

The output from the random projections is again a (denser) vector of -1s and +1s. Because this carries much less information than the real-valued vectors from the minhashing, the banding technique does not work--too many items would have the same band. So, what does the SparseLSH apply then? What do they write in the book? What does wikipedia say?
- see [here, p. 58](https://web.stanford.edu/class/cs246/slides/04-lsh_theory.pdf): they apply the bands technique to the Euclidean distance

[continue in pdf]

section 3.6: family of locality-senstive functions
- minhash function is one family of locality-sensitive functions
- 


So, plan for 16/01
- make toy example with dot product 
- check timing
- should be relatively easy to integrate into REL/existing class?

procedure
1. ie, generate sparse matrix of dimension (length_signature, length(vocabulary)). one row corresponds to one hash function
2. generate dot product of each row with the vectors (assign -1/1 depending on sign). [check again: where do they np bitpack? could this be useful here or not?]
    - the output is one entry in the signature
3. then use banding like I currently use. 

What does SparseLSH do?
- key question: at query time, I think this transforms the keys back to sparse -- could also take a lot of memory? 
    - maybe querying one by one in REL would be an option, but that depends on how long these queries take!
    - the `query` function seems to do quite some looping, so I am not sure how fast it is.. 
    - the advantage of the minhashing is that it returns the similarity of the items in one go (although very slowly for large inputs)
    - can I profile their function and see if it scales well (although the output seems somewhat random at the moment)
    - "You do not save processing by limiting the results. Currently, a similarity ranking and sort is done on all items in the hashtable."
        - where do they do the sort? 
- there must be some methods that are implemented in the `storage` class
    - `get_list`?
    - `append`? 
    - see below. basic dict operations suffice for the current use case. 
- at init of class:
    - `_generate_random_planes`: "uniformly distributed hyperplanes" (but they use `np.random.randn`, which seems to wrap around `np.standard_normal`?)
    - `_init_hashtables`: stores the hash tables in `storage`. if in-memory, then I think it is just a dict. 
- main steps:
    ```python
    lsh = SparseLSH() # init
    lsh.index(X) # X is the sparse binary matrix
    lsh.query()
    ```
- `_hash`: generate binary hashes for input points. Ie:
    ```python
    def _hash(self, planes, input_points): # do we do this for all hyperplanes??
        planes = planes.transpose()
        projections = input_points.dot(planes)
        signs = (projections > 0) # are these the sketches?
        return  np.packbits(signs.toarray(), axis=-1) #  Packs the elements of a binary-valued array into bits in a uint8 array.
    ```
- `index()` function
    ```python
    def index(self, input_points, extra_data=None): # s
        " Index input points by adding them to the selected storage."
        # assume extra_data is always `None`.
        for i, table in enumerate(self.hash_tables):
            keys = self._hash(self.uniform_planes[i])

            for j in range(keys.shape[0]): # not sure what this does
                value = (input_points[j],)
                table.append_val(keys[j].tobytes(), value) # what does append_val do? I guess it adds the data to the storage. in the present case, just use memory 
    ```
- `query()` function
    - for a distance function, return the closests candidates
    - matches exactly on one of the hash keys 
    ```python
    candidates = []
    for i, table in enumerate(self.hash_tables):
        # get hashes of query points for the specific plane
        keys = self._hash(self.uniform_planes[i], query_points)
        for j in range(keys.shape[0]):
            # Create a sublist of candidate neighbors for each query point
            if len(candidates) <= j:
                candidates.append([])
            new_candidates = table.get_list(keys[j].tobytes())
            if new_candidates is not None and len(new_candidates) > 0:
                candidates[j].extend(new_candidates) # what does extend do? 
        # some parts are skipped .. 
        distances = d_func(query_points[j], cand_csr) # main function. how big is cand_csr? this could be very expensive if cand_csr is large 
    ```

this is the class for in-memory storage. these are just a bunch of operations on dictionaries. how exactly are the data stored?

```python
class InMemoryStorage(BaseStorage):
    def __init__(self, config):
        self.name = 'dict'
        self.storage = dict()

    def keys(self):
        return list(self.storage.keys())

    def set_val(self, key, val):
        self.storage[key] = val

    def get_val(self, key):
        return self.storage[key]

    def append_val(self, key, val):
        self.storage.setdefault(key, []).append(val) # what does setdefault do? 

    def get_list(self, key):
        return self.storage.get(key, [])

```

In [6]:
a = np.array([5,6,-1,6,9,-11])
signs = (a > 0)
signs

np.packbits(signs, axis=-1)

array([216], dtype=uint8)

In [1]:
from load_coreferences import load_coreferences
# import lsh 
import REL # install with pip install -e ../REL/.
import copy

import faiss 
import numpy as np
from REL import lsh 


0. Load data 

In [2]:
raw_mentions = load_coreferences(drop_duplicates=False)


In [3]:
len(raw_mentions)

632

In [3]:
mentions = {i: m for i, m in enumerate(raw_mentions)}
# stack them on top of each other 

mentions_scaled = copy.copy(mentions)

idx = len(mentions_scaled)
scaling_factor = 5
for i in range(1, scaling_factor):
    for idx_old in mentions.keys():
        m = mentions[idx_old]
        mentions_scaled[idx] = m 
        idx += 1

## MinHashing with numpy

In [6]:
len(mentions_scaled)

mylsh = lsh.LSHMinHash(mentions=mentions_scaled, shingle_size=2, signature_size=200, band_length=10)

# mylsh.cluster(numpy_signature=True)
# mylsh.summarise()

mylsh.cluster(numpy_signature=False)
mylsh.summarise()

took 1.1436002254486084 seconds for 3160 mentions
average, min, max cluster size: 187.92, 14, 379



## Notes for using numpy 
- what is the exact time complexity here? certainly more than linear. check the theory.
- (probably) memory issues when making the signature with numpy


### How to integrate with REL?

In [7]:
# out = {i: {"shingles": lsh.k_shingle(m, 4)} for i, m in zip(range(len(mentions_rel), mentions)) }
# mentions_rel
mentions_rel = [
    'German', 'British', 'Brussels', 'European Commission', 'German',
    'British', 'Germany', 'European Union', 'Britain', 'Commission', 
    'European Union', 'Franz Fischler', 'Britain', 'France', 'BSE', 
    'Spanish', 'Loyola de Palacio', 'France', 'Britain', 'BSE', 'British', 'German', 
    'British', 'Europe', 'Germany', 'Bonn', 'British', 'Germany', 'Britain', 'British'
]

mylsh = lsh.LSHMinHash(mentions=mentions_rel, shingle_size=4, signature_size=50, band_length=2)
mylsh.cluster()
mylsh.summarise()



took 0.0069942474365234375 seconds for 30 mentions
average, min, max cluster size: 29.0, 29, 29


How to deal with input where only one item is repeated? -- put as a test!

In [8]:
import lsh 
import numpy as np

test_mentions = ['EEC', 'EEC'] # this fails. ['German', 'German'] does not. neither does ['EEC', 'EEC', 'German']
test_mentions = ['EEC', 'ABC'] # this also fails. ['EEC', 'ABCde'] does not. 
# the problem is that when there is no mention of at least the inputted shingle size, the binary vectors cannot be calculated

mylsh = lsh.LSHMinHash(mentions=test_mentions, shingle_size=4, signature_size=300, band_length=2)

mylsh._build_vocab()
mylsh.encode_binary(to_numpy=True)

mylsh.vectors

mylsh.vectors.shape[1] # this should not be 0
mylsh.vectors.shape
# mylsh.make_signature()
mylsh.cluster()
mylsh.candidates




[{0, 1}, {0, 1}]

## with FAISS

In [6]:

mylsh = lsh.LSHBitSampling(mentions=mentions_scaled, shingle_size=4)

mylsh.faiss_neighbors(k=10, nbits=100)
neighbors = mylsh.neighbors_to_dict()
mylsh.summarise() # note: 50% of the time here is taken for extracting the neighbors! can this be made faster? ie with numpy?



# note: this approach uses quite some memory because it stores the full vectors and shingles.. change??
# here we see the problem -- there are 20 duplicates of any mention, but the 10 nearest neighbors are all in the same bucket
    # how to avoid? we do not know the number of duplicates beforehand... count? somehow create unique mentions to avoid this? 

Took 0.34625840187072754 seconds to classify 3160 mentions


Notes 
- It is important to use the binary vectors. If using the min-hashed vectors, longer words are closer to longer ones (and shorter to shorter)
- higher k -> higher recall
    - low k may not capture the actual coreferences. ie, for k=4, it missed them for "eagles", "rosati", "belo"
    - using k=10 fixes the problems for "eagles" and "belo" but not for "rosati"


Next steps
- scaling -- seems very good 
- precision/recall of whole pipeline (including the search for coreferences)
- how easy is it to integrate FAISS into REL?
- a problem could be when there are a lot of mentions that refer to the same (single) mention. ie, 10 times "jimi" and once "jimi hendrix". 
    - I think we would miss "jimi hendrix" for k=10
    - Solutions: larger k? -> quadratic cost again 
    - Restrict the hamming distance of the neighbors, ie forbid them to be in the same bucket (this would also omit the own element already I think)
    - do some more testing for this 
    - or just use the unique mentions






## new approach with random projections

In [5]:
from scipy import sparse 

raw_mentions
mylsh = lsh.LSHMinHash(mentions=raw_mentions, shingle_size=4, signature_size=50, band_length=2)

# mylsh._build_vocab()
# mylsh.encode_binary()
# vectors = sparse.csr_matrix(mylsh.vectors)

# mylsh.make_signature()

mylsh.cluster()



In [6]:

length_signature = 40
rng = np.random.default_rng(seed=3)
vectors = mylsh.vectors
hyperplanes = rng.choice([-1, 1], (length_signature, vectors.shape[1]))
hyperplanes = sparse.csr_matrix(hyperplanes)

hyperplanes[0, :].shape

(1, 880)

In [17]:

plane = hyperplanes[1, :]
display(plane.shape)
display(vectors.shape)
display(plane.transpose().shape)
out = vectors.dot(plane.transpose())
out = out.toarray()

sig_i = (out > 0)

sig_i = sig_i.astype(int)

1 + sig_i

test = vectors.dot(hyperplanes.transpose()) #.shape
test = test.toarray()
test = (test > 0)
1 + test

(1, 880)

(632, 880)

(880, 1)

array([[2, 2, 2, ..., 1, 2, 2],
       [1, 2, 2, ..., 2, 2, 1],
       [2, 2, 2, ..., 1, 2, 2],
       ...,
       [2, 1, 2, ..., 2, 1, 2],
       [2, 1, 2, ..., 1, 1, 2],
       [2, 1, 2, ..., 2, 1, 2]])

## with MinHashing (does not scale well, and not work at the moment)

In [8]:
len(mentions_scaled)

mylsh = lsh.LSHMinHash_nonp(mentions=mentions_scaled, shingle_size=3, signature_size=200, n_buckets=2)

mylsh.cluster()
mylsh.summarise()

# adjust the sizing according to the rules so that we have log-time complexity!

took 49.10193204879761 seconds for 3480 mentions
average, min, max cluster size: 78.08, 39, 259


## Create signature with numpy -- main function to put to lsh

In [7]:
def reshape_rows_reps(a):
    "reshape a 3-d array of n_reps x n_rows x n_cols to n_rows x n_reps x n_cols"
    n_reps, n_rows, n_cols = a.shape
    a = a.reshape(n_reps*n_rows, n_cols)
    # extractor indices: for 3 reps, 2 rows: [0,2,4,1,3,5]. to reorder a
        # in other words: goes from 0 to (n_reps * n_rows). step sizes are n_rows. starts are the row indices
    idx = np.arange(n_reps*n_rows).reshape(n_reps, n_rows).T.reshape(-1,1)
    a = np.take_along_axis(a, idx, axis=0)
    a = a.reshape(n_rows, n_reps, n_cols)
    return a 

def minhash_signature_np(x, n_reps):
    """Make a minhash signature of array x with length n_reps.

    Inputs
    ------
    x: axis 0 are observations, columns are binary one-hot encoded vectors
    """
    # get indices 
    indices = np.arange(x.shape[1])
    rng = np.random.default_rng(12345)

    # expand by n_reps 
    indices_mult = np.tile(indices, (n_reps, 1)) # reorder the columns n_reps times 
    x_mult = np.tile(x, (n_reps, 1)).reshape((n_reps,) + x.shape) # new shape: (n_resp, x.shape[0], x.shape[1

    # permute indices and apply to x_mult
    permuted_indices = rng.permuted(indices_mult, axis=1)
    x_mult_permuted = np.take_along_axis(x_mult, permuted_indices[:, np.newaxis], 2)

    # for the reduction below, need to have all samples of the same observation in one block
    x_mult_permuted = reshape_rows_reps(x_mult_permuted)

    # make signature
    sig = x_mult_permuted.argmax(axis=2)
    return sig 

In [8]:
## simulate input 
n_cols = 4 # (size of dict?)
n_rows = 2 # number of rows in original array (mnumber of mentions)
rng = np.random.default_rng(12345)
x = rng.standard_normal(size=n_rows*n_cols).reshape(n_rows, n_cols)
# x = rng.choice(range(2), size=n_rows*n_cols, p=[0.9, 0.1]).reshape(n_rows, n_cols)
x

array([[-1.42382504,  1.26372846, -0.87066174, -0.25917323],
       [-0.07534331, -0.74088465, -1.3677927 ,  0.6488928 ]])

In [9]:

minhash_signature_np(x, 3)
    

array([[3, 3, 1],
       [2, 2, 0]])

# AAR

Here are a couple of links for this re-sorting
- https://stackoverflow.com/questions/35646908/numpy-shuffle-multidimensional-array-by-row-only-keep-column-order-unchanged
- https://stackoverflow.com/questions/70683286/fastest-way-to-sample-many-random-permutations-of-a-numpy-array
- https://stackoverflow.com/questions/20265229/rearrange-columns-of-numpy-2d-array
- https://stackoverflow.com/questions/53421637/reorganizing-a-3d-numpy-array
- https://stackoverflow.com/questions/64235838/how-to-sort-each-row-of-a-3d-numpy-array-by-another-2d-array

### How to order a 3-dimensional array along one axis? -- put to general AAR

- important: the input array and the ordering array need to have the same dimensions when using `np.take_along_axis`

In [9]:
# one dimension
x = np.array([10, 20, 30, 40])
print(f"x: {x}")
p = np.array([3,0,1,2])
print(f"x reordered: {x[p]}")
# two dimensions
x = np.array([[10, 20, 30, 40],[10, 20, 30, 40]] )
print(f"x:")
display(x)
print(f"x reordered:")
display(x[:, p])
# three dimensions
x = np.array([[10, 20, 30, 40],[10, 20, 30, 40]])[:, np.newaxis]
print("x:")
display(x)
print("x reordered:")
display(x[:, :, p])
# display(x)

# # now add another dimension to p 
p2 = np.array([[3,0,1,2], [2,0,3,1]])
display(p2[:, np.newaxis].shape)
display(x.shape)
np.take_along_axis(x, p2[:, np.newaxis], 2)

# now add expand dimension 1 of x 
x = np.array([[[10, 20, 30, 40], [5,8,9,3]],[[10, 20, 30, 40], [5,8,9,3]]] )
print("x:")
display(x)
print("x reordered:")
np.take_along_axis(x, p2[:, np.newaxis], 2)




x: [10 20 30 40]
x reordered: [40 10 20 30]
x:


array([[10, 20, 30, 40],
       [10, 20, 30, 40]])

x reordered:


array([[40, 10, 20, 30],
       [40, 10, 20, 30]])

x:


...

(2, 1, 4)

x:


array([[[10, 20, 30, 40],
        [ 5,  8,  9,  3]],

       [[10, 20, 30, 40],
        [ 5,  8,  9,  3]]])

x reordered:


array([[[40, 10, 20, 30],
        [ 3,  5,  8,  9]],

       [[30, 10, 40, 20],
        [ 9,  5,  3,  8]]])

### Another application: reorder the rows -- how is that different from the above? -- Put to AAR!

In [10]:
# start again with a simple example
x = np.array([[1,2],[3,4]])
idx = np.array([1, 0])
x[idx, :]

## three dimensions
x = np.array([[10, 20, 30, 40],[8, 4, 8, 1]])[:, np.newaxis]
print("x:")
display(x)
print("reordered x:")
display(x[idx, :])


## three dimensions, multiple rows
x = np.array([
    [[10, 20, 30, 40], [5,8,9,3]], # [0, 0, 0, 0]],
    [[10, 20, 30, 40], [5,8,9,3]]#, [0, 0, 0, 0]] # this also does not work 
    ] )
print("x:")
display(x)

reordered_idx = np.array([[0, 0], [1, 1]]).reshape(2,2,1)
display(reordered_idx)

np.take_along_axis(x, reordered_idx, 1)


x:


array([[[10, 20, 30, 40]],

       [[ 8,  4,  8,  1]]])

reordered x:


array([[[ 8,  4,  8,  1]],

       [[10, 20, 30, 40]]])

x:


array([[[10, 20, 30, 40],
        [ 5,  8,  9,  3]],

       [[10, 20, 30, 40],
        [ 5,  8,  9,  3]]])

array([[[0],
        [0]],

       [[1],
        [1]]])

array([[[10, 20, 30, 40],
        [10, 20, 30, 40]],

       [[ 5,  8,  9,  3],
        [ 5,  8,  9,  3]]])

In [11]:
# more reps -- TODO: test case 1
x = np.array([ 
    [[10, 20, 30, 40], [5,8,9,3]],
    [[10, 20, 30, 40], [5,8,9,3]],
    [[10, 20, 30, 40], [5,8,9,3]]
    ])
print("x:")
display(x)

print("reordered x:")
display(reshape_rows_reps(x))


x:


array([[[10, 20, 30, 40],
        [ 5,  8,  9,  3]],

       [[10, 20, 30, 40],
        [ 5,  8,  9,  3]],

       [[10, 20, 30, 40],
        [ 5,  8,  9,  3]]])

reordered x:


array([[[10, 20, 30, 40],
        [10, 20, 30, 40],
        [10, 20, 30, 40]],

       [[ 5,  8,  9,  3],
        [ 5,  8,  9,  3],
        [ 5,  8,  9,  3]]])

In [12]:
# more rows -- TODO: test case 2
x = np.array([
    [[10, 20, 30, 40], [5,8,9,3], [0, 0, 0, 0]],
    [[10, 20, 30, 40], [5,8,9,3], [0, 0, 0, 0]] # this also does not work 
    ] )
print("x:")
display(x)

print("reordered x:")
display(reshape_rows_reps(x))

x:


array([[[10, 20, 30, 40],
        [ 5,  8,  9,  3],
        [ 0,  0,  0,  0]],

       [[10, 20, 30, 40],
        [ 5,  8,  9,  3],
        [ 0,  0,  0,  0]]])

reordered x:


array([[[10, 20, 30, 40],
        [10, 20, 30, 40]],

       [[ 5,  8,  9,  3],
        [ 5,  8,  9,  3]],

       [[ 0,  0,  0,  0],
        [ 0,  0,  0,  0]]])

```python 
# https://numpy.org/doc/stable/reference/generated/numpy.take_along_axis.html
Ni, M, Nk = a.shape[:axis], a.shape[axis], a.shape[axis+1:]
# let a be an array with n_reps of n_rows rows with n_cols cols. a.shape = (n_reps, n_rows, n_cols)
# Ni: length of all dimensions until axis. in my case, [n_reps]
# M: n_rows 
# Nk: n_cols
J = indices.shape[axis]  # Need not equal M
# J: n_reps
out = np.empty(Ni + (J,) + Nk) 

for ii in ndindex(Ni):
    for kk in ndindex(Nk):
        a_1d       = a      [ii + s_[:,] + kk]
        indices_1d = indices[ii + s_[:,] + kk]
        out_1d     = out    [ii + s_[:,] + kk]
        for j in range(J):
            out_1d[j] = a_1d[indices_1d[j]]
# output: Ni x J x Nk = n_reps x n_reps x n_cols. but what I want is n_rows x n_reps x n_cols (?) thus, I need to change the input, ie to change the shape of a.
```

In [13]:
display(x)
xb = x.reshape(3,8) # 3 = number of reps, 8 = n_col * n_rows
out = np.stack(np.split(xb,2,axis=1)) # 2 = n_rows
out # this works, but is it efficient?
# https://stackoverflow.com/questions/21580693/numpy-array-split-partition-efficiency

array([[[10, 20, 30, 40],
        [ 5,  8,  9,  3],
        [ 0,  0,  0,  0]],

       [[10, 20, 30, 40],
        [ 5,  8,  9,  3],
        [ 0,  0,  0,  0]]])

array([[[10, 20, 30, 40],
        [ 0,  0,  0,  0],
        [ 5,  8,  9,  3]],

       [[ 5,  8,  9,  3],
        [10, 20, 30, 40],
        [ 0,  0,  0,  0]]])

### Draw multiple samples of a given array where the columns are randomly reordered 

In [14]:
rng = np.random.default_rng(1234)
x = np.array([[1,2,3, 10], [4,5,6,20]]) # the starting array 
display(x)

indices = np.arange(x.shape[1])
n_reps = 3
indices_mult = np.tile(indices, (n_reps, 1)) # reorder the columns n_reps times 
x_mult = np.tile(x, (n_reps, 1)).reshape((n_reps,) + x.shape) # new shape: (n_resp, x.shape[0], x.shape[1

display(indices_mult)
# permute 
permuted_indices = rng.permuted(indices_mult, axis=1)
display(permuted_indices)

# reorder the sample
np.take_along_axis(x_mult, permuted_indices[:, np.newaxis], 2)

array([[ 1,  2,  3, 10],
       [ 4,  5,  6, 20]])

array([[0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3]])

array([[0, 3, 1, 2],
       [1, 2, 3, 0],
       [1, 3, 0, 2]])

array([[[ 1, 10,  2,  3],
        [ 4, 20,  5,  6]],

       [[ 2,  3, 10,  1],
        [ 5,  6, 20,  4]],

       [[ 2, 10,  1,  3],
        [ 5, 20,  4,  6]]])

### Permuting columns: compare approaches

In [15]:

rng = np.random.default_rng(1234)
# x is big enough to not want to enumerate all permutations
x = rng.standard_normal(size=20)
n = 10000
perms = np.array([rng.permutation(x) for _ in range(n)])
perms 

def list_comp(x, n):
    rng_temp = np.random.default_rng(1)
    perms = np.array([rng_temp.permutation(x) for _ in range(n)])
    return perms 


def while_loop(x, n):
    rng_temp = np.random.default_rng(1)
    i = 0 
    templist = []
    while i < n:
        templist.append(rng_temp.permutation(x))
        i += 1
    return np.stack(templist) 

def np_permuted(x, n):
    rng_temp = np.random.default_rng(1)
    perms = rng_temp.permuted(np.tile(x, n).reshape(n,x.size), axis=1)
    return perms 

out1 = list_comp(x, n)
out2 = while_loop(x, n)
out3 = np_permuted(x, n)

assert not np.any(out1 != out2)
assert not np.any(out1 != out3)

%timeit list_comp(x, n)
%timeit while_loop(x, n)
%timeit np_permuted(x, n)

20.7 ms ± 318 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
23.6 ms ± 394 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.64 ms ± 24.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
